In [1]:
import math
import numpy as np
import pandas as pd

def generate_adjacency_matrix(N):
    # n_genes = 10
    a = np.random.randint(0, 3, (N, N))
    a[np.tril_indices(a.shape[0], -1)] = a.T[np.tril_indices(a.shape[0], -1)]
    b = np.fliplr(a)
    b[np.tril_indices(b.shape[0], -1)] = b.T[np.tril_indices(a.shape[0], -1)]

    b[b == 2] = -1
    np.fill_diagonal(b, 0)  # Remove self loops
    return b

def get_vector_that_provides_corr(rho, x1):
    n = x1.shape[0]  # length of vector, the number of samples for the same gene that we define in n_cols
    # ho = 0.6  # desired correlation = cos(angle)
    theta = math.acos(rho)  # corresponding angle
    x2 = np.random.normal(2, 0.5, n)  # new random data
    X = np.c_[x1, x2]  # matrix
    Xctr = pd.DataFrame(X).apply(lambda x: x - x.mean()).to_numpy()  # centered columns (mean 0)
    Id = np.diag(np.repeat(1, n))  # identity matrix
    Q = np.linalg.qr(Xctr[:, 0].reshape(n, 1))[0].flatten()  # QR-decomposition, just matrix Q
    P = np.outer(Q, Q)  # projection onto space defined by x1
    x2o = np.dot((Id - P), Xctr[:, 1])  # x2ctr made orthogonal to x1ctr
    Xc2 = np.c_[Xctr[:, 0], x2o]  # bind to matrix
    Y = np.dot(Xc2, np.diag(1 / np.sqrt(np.sum(np.power(Xc2, 2), axis=0))))  # scale columns to length 1
    x = Y[:, 1] + (1 / np.tan(theta)) * Y[:, 0]
    x_norm = (x - x.mean()) / (x.std())
    return x_norm


def simulate_expression_from_adjacency(adjamat, n_samples=100, simdata_noise=0.1):
    # noise = 0.5 # .1 strong correlation, 3 weak correlation
    # random initialize data matrix
    # n_samples = 40 # number of samples: 40 so we later split into 20 cases and 20 controls
    n_rows = adjamat.shape[0]  # number of genes
    mu, sigma = 1, 1  # mean and standard deviation
    dataset = np.random.normal(mu, sigma, n_samples * n_rows).reshape(n_rows, n_samples)
    already_modified = np.repeat(0, n_rows)  # N=n1+n2 genes
    already_modified[0] = 1  # leave the first gene alone, base case

    for i in range(adjamat.shape[1]):
        for j in range(i + 1, adjamat.shape[1]):
            # print(f'Considering row: {i}, column: {j} of A')
            if (adjamat[i, j] == 1) & (already_modified[j] == 0):
                # print(i,j)
                # print(A[i,j])
                dataset[j, :] = get_vector_that_provides_corr(rho=0.999999, x1=dataset[i, :]) + \
                                np.random.normal(0, simdata_noise, dataset.shape[1])
                already_modified[j] = 1
            elif (adjamat[i, j] == -1) & (already_modified[j] == 0):
                # print(i,j)
                # print(A[i,j])
                dataset[j, :] = get_vector_that_provides_corr(rho=-0.999999, x1=dataset[i, :]) + \
                                np.random.normal(0, simdata_noise, dataset.shape[1])
                already_modified[j] = 1
    ds = pd.DataFrame(dataset)
    ds.columns = 'sample_' + ds.columns.astype(str)
    ds.index = 'gene_' + ds.index.astype(str)
    return ds

def ground_truth_from_adjamat(adjamat):
    genenames = ['gene_' + s for s in list(str(i) for i in range(n_genes))]

    gene_A = []
    gene_B = []
    rels = []
    for i in range(adjamat.shape[1]):
        for j in range(i + 1, adjamat.shape[1]):
            if adjamat[i][j] != 0:
                #print(i, j, adjamat[i, j])
                gene_A.append(i)
                gene_B.append(j)
                rels.append(np.where(adjamat[i, j] > 0, "[+]", "[-]"))

    pairs = pd.DataFrame({'gene_A': gene_A, 'gene_B': gene_B, 'correlation': rels})
    mygenedict = dict(zip(range(adjamat.shape[1]), genenames))
    pairs.gene_A = pairs.gene_A.map(mygenedict)
    pairs.gene_B = pairs.gene_B.map(mygenedict)
    return pairs

In [2]:
n_genes = 10
adjamat = generate_adjacency_matrix(N=n_genes)

ds = simulate_expression_from_adjacency(adjamat=adjamat, n_samples=100, simdata_noise=0.1)
groundtruth = ground_truth_from_adjamat(adjamat)
#ds.to_csv('dataset_'+ str(n_genes) + '.csv')
print(ds)
print(groundtruth)

        sample_0  sample_1  sample_2  sample_3  sample_4  sample_5  sample_6  \
gene_0 -1.265224 -0.630353  0.048640  3.608028  2.080957  2.532421  2.072899   
gene_1 -2.036149 -1.628028 -0.752407  2.466489  1.255533  1.355782  0.999540   
gene_2 -2.119507 -1.401712 -0.753136  2.551723  0.874884  1.645693  1.198898   
gene_3  1.038377 -0.058356  0.662654  0.333821  1.409789  1.006426  0.184323   
gene_4 -2.120642 -1.489312 -0.883486  2.494029  1.159912  1.534994  1.142582   
gene_5 -2.032378 -1.348682 -0.675889  2.391988  1.348645  1.566360  1.195850   
gene_6 -2.047729 -1.604410 -0.888245  2.560301  1.136988  1.461081  1.037619   
gene_7 -2.162308 -1.369710 -0.893040  2.353393  1.177512  1.472979  1.181590   
gene_8 -2.058287 -1.507532 -0.875531  2.558591  1.058328  1.571144  1.045854   
gene_9  2.099635  1.404928  0.960661 -2.506998 -0.691187 -1.610860 -1.154508   

        sample_7  sample_8  sample_9  ...  sample_90  sample_91  sample_92  \
gene_0  0.733625  1.332743  0.056054  ...